In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,SimpleRNN
import tensorflow as tf


In [2]:
df = pd.read_csv(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\featue_engg_data1.csv")
#df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Season_Summer,Season_Winter,Demand_Prev_Hour,Demand_Yesterday_Same_Hour,Demand_Last_Week_Same_Hour,Rolling_Mean_3H,Rolling_Mean_24H,Total_Gen,Renewable_Pct,Fossil_Pct
0,2018-07-01 17:00:00,12.5,28976.042,21138.304,-6528.677,28386.220,8.764,7725.951,2261.073,3089.741,...,1,0,31560.368,97906.322,30623.583,52909.261,42604.509,17521.203,42.950,44.145
1,2018-07-01 18:00:00,12.5,29065.500,21217.885,-6554.521,28486.010,8.792,7761.917,2261.062,3110.594,...,1,0,19035.781,36973.722,30187.366,26585.677,41847.195,17577.073,42.927,44.209
2,2018-07-01 19:00:00,12.5,29154.958,21297.467,-6580.365,28585.800,8.819,7797.882,2261.052,3131.446,...,1,0,120405.354,28886.583,96196.009,60325.969,45629.697,17632.943,42.904,44.273
3,2018-07-01 20:00:00,12.5,29244.417,21377.049,-6606.208,28685.590,8.847,7833.847,2261.042,3152.299,...,1,0,8629.271,31655.750,35642.181,23684.576,48609.446,17688.812,42.881,44.337
4,2018-07-01 21:00:00,12.5,29333.875,21456.630,-6632.052,28785.380,8.875,7869.812,2261.031,3173.151,...,1,0,31562.326,29065.500,8211.854,30046.873,49142.567,17744.682,42.858,44.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-29 20:00:00,12.5,64967.778,65025.979,58.042,65732.083,8989.486,27552.910,4647.181,32.896,...,1,0,40554.444,14324.708,37711.542,67747.671,49409.849,46077.069,10.607,79.307
525056,2023-06-29 21:00:00,12.5,64902.135,64963.474,61.177,65699.302,8993.729,27593.401,4647.292,33.224,...,1,0,31739.526,22046.601,65821.128,37511.137,46137.266,46106.208,10.566,79.354
525057,2023-06-29 22:00:00,12.5,64836.493,64900.969,64.312,65666.521,8997.972,27633.892,4647.403,33.552,...,1,0,15892.381,31801.521,21450.109,39168.139,45454.091,46135.347,10.526,79.401
525058,2023-06-29 23:00:00,12.5,64770.851,64838.464,67.448,65633.740,9002.215,27674.384,4647.514,33.880,...,1,0,22263.507,97828.229,41419.396,42477.179,39779.804,46164.486,10.485,79.448


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# 1. Utility functions
def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

# 2. Read dataset
df = pd.read_csv(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\featue_engg_data1.csv")
df['Date'] = pd.to_datetime(df['Date'])

# 3. Loop over regions
results = []
for region_code, region_name in [('CAL', 'California'), ('TEX', 'Texas')]:
    region_df = df[df['Region'] == region_code].copy()
    region_df.set_index('Date', inplace=True)
    daily_data = region_df.groupby(region_df.index).agg({
        'Demand': 'sum',
        'CO2_Total_Emissions': 'sum',
        'Hour': 'mean',
        'Month': 'mean',
        'DayOfWeek': 'mean',
        'Is_Weekend': 'mean',
        'DayOfYear': 'mean',
        'WeekOfYear': 'mean',
        'Season_Autumn': 'mean',
        'Season_Spring': 'mean',
        'Season_Summer': 'mean',
        'Season_Winter': 'mean',
        'Renewable_Pct': 'mean',
        'Fossil_Pct': 'mean',
        'Demand_Prev_Hour': 'mean',
        'Demand_Yesterday_Same_Hour': 'mean',
        'Demand_Last_Week_Same_Hour': 'mean',
        'Rolling_Mean_3H': 'mean',
        'Rolling_Mean_24H': 'mean'
    })

    features = daily_data[[  # Same features
        'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
        'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
        'Renewable_Pct', 'Fossil_Pct',
        'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
        'Rolling_Mean_3H', 'Rolling_Mean_24H'
    ]]
    targets = daily_data[['Demand', 'CO2_Total_Emissions']]

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(features)
    y_scaled = scaler_y.fit_transform(targets)

    X_seq, y_seq = create_sequences(X_scaled, y_scaled)
    lookback = 30
    split = int(0.8 * len(X_seq))
    X_train, X_test = X_seq[:split], X_seq[split:]
    y_train, y_test = y_seq[:split], y_seq[split:]

    model = Sequential([
        SimpleRNN(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)
    test_dates = daily_data.index[lookback + split:]

    # Compute metrics
    result = {
        'Region': region_name,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2': mape(y_true[:, 1], y_pred[:, 1])
    }
    results.append(result)

    # Save results as DataFrame
    rnn_results_df = pd.DataFrame({
        'Date': test_dates,
        'Region': region_name,
        'Model': 'RNN',
        'Actual_Demand': y_true[:, 0],
        'Predicted_Demand': y_pred[:, 0],
        'Actual_CO2': y_true[:, 1],
        'Predicted_CO2': y_pred[:, 1]
    })
    rnn_results_df.to_csv(f'rnn_{region_name.lower()}_forecast.csv', index=False)
    print(f"Saved results for {region_name}")

# Show results table
results_df = pd.DataFrame(results)
print('\nRNN Forecasting Results:')
print(results_df.round(2))


Epoch 1/10


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0413
Epoch 2/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0057
Epoch 3/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0049
Epoch 4/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0042
Epoch 5/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0041
Epoch 6/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0040
Epoch 7/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0038
Epoch 8/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0038
Epoch 9/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0036
Epoch 10/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0035
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Saved results for California
Epoch 1/10


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0433
Epoch 2/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0058
Epoch 3/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0052
Epoch 4/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0050
Epoch 5/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0045
Epoch 6/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0044
Epoch 7/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0042
Epoch 8/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0041
Epoch 9/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0040
Epoch 10/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0038
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Saved results for Texas

RNN Forecasting Results:
       Region  MAE_Demand  RMSE_Demand  MAPE_Demand  MAE_CO2  RMSE_CO2  \
0  California     1191.49      1703.24         3.81   592.82    759.00   
1       Texas     2581.46      3362.35         5.29  1430.50   1821.62   

   MAPE_CO2  
0     10.53 

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# 1. Utility functions
def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

# 2. Read dataset
df = pd.read_csv(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\featue_engg_data1.csv")
df['Date'] = pd.to_datetime(df['Date'])

# 3. Loop over regions
results = []
for region_code, region_name in [('CAL', 'California'), ('TEX', 'Texas')]:
    region_df = df[df['Region'] == region_code].copy()
    region_df.set_index('Date', inplace=True)
    daily_data = region_df.groupby(region_df.index).agg({
        'Demand': 'sum',
        'CO2_Total_Emissions': 'sum',
        'Hour': 'mean',
        'Month': 'mean',
        'DayOfWeek': 'mean',
        'Is_Weekend': 'mean',
        'DayOfYear': 'mean',
        'WeekOfYear': 'mean',
        'Season_Autumn': 'mean',
        'Season_Spring': 'mean',
        'Season_Summer': 'mean',
        'Season_Winter': 'mean',
        'Renewable_Pct': 'mean',
        'Fossil_Pct': 'mean',
        'Demand_Prev_Hour': 'mean',
        'Demand_Yesterday_Same_Hour': 'mean',
        'Demand_Last_Week_Same_Hour': 'mean',
        'Rolling_Mean_3H': 'mean',
        'Rolling_Mean_24H': 'mean'
    })

    features = daily_data[[  # Same features
        'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
        'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
        'Renewable_Pct', 'Fossil_Pct',
        'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
        'Rolling_Mean_3H', 'Rolling_Mean_24H'
    ]]
    targets = daily_data[['Demand', 'CO2_Total_Emissions']]

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(features)
    y_scaled = scaler_y.fit_transform(targets)

    X_seq, y_seq = create_sequences(X_scaled, y_scaled)
    lookback = 30
    split = int(0.75 * len(X_seq))
    X_train, X_test = X_seq[:split], X_seq[split:]
    y_train, y_test = y_seq[:split], y_seq[split:]

    model = Sequential([
        SimpleRNN(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)
    test_dates = daily_data.index[lookback + split:]

    # Compute metrics
    result = {
        'Region': region_name,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2': mape(y_true[:, 1], y_pred[:, 1])
    }
    results.append(result)

    # Save results as DataFrame
    rnn_results_df = pd.DataFrame({
        'Date': test_dates,
        'Region': region_name,
        'Model': 'RNN',
        'Actual_Demand': y_true[:, 0],
        'Predicted_Demand': y_pred[:, 0],
        'Actual_CO2': y_true[:, 1],
        'Predicted_CO2': y_pred[:, 1]
    })
    rnn_results_df.to_csv(f'rnn_{region_name.lower()}_forecast.csv', index=False)
    print(f"Saved results for {region_name}")

# Show results table
results_df = pd.DataFrame(results)
print('\nRNN Forecasting Results:')
print(results_df.round(2))


Epoch 1/10


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


513/513 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0155
Epoch 2/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0050
Epoch 3/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0044
Epoch 4/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0043
Epoch 5/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0039
Epoch 6/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0039
Epoch 7/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0035
Epoch 8/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0035
Epoch 9/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0033
Epoch 10/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0031
342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Saved results for California
Epoch 1/10


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


513/513 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0570
Epoch 2/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0059
Epoch 3/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0053
Epoch 4/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0050
Epoch 5/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0047
Epoch 6/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0046
Epoch 7/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0043
Epoch 8/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0042
Epoch 9/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0039
Epoch 10/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0038
342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Saved results for Texas

RNN Forecasting Results:
       Region  MAE_Demand  RMSE_Demand  MAPE_Demand  MAE_CO2  RMSE_CO2  \
0  California     1316.87      1813.52         4.12   582.79    730.21   
1       Texas     3267.01      4075.21         6.32  1463.40   1757.34   

   MAPE_CO2  
0     11.00 

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# 1. Utility functions
def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

# 2. Read dataset
df = pd.read_csv(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\featue_engg_data1.csv")
df['Date'] = pd.to_datetime(df['Date'])

# 3. Loop over regions
results = []
for region_code, region_name in [('CAL', 'California'), ('TEX', 'Texas')]:
    region_df = df[df['Region'] == region_code].copy()
    region_df.set_index('Date', inplace=True)
    daily_data = region_df.groupby(region_df.index).agg({
        'Demand': 'sum',
        'CO2_Total_Emissions': 'sum',
        'Hour': 'mean',
        'Month': 'mean',
        'DayOfWeek': 'mean',
        'Is_Weekend': 'mean',
        'DayOfYear': 'mean',
        'WeekOfYear': 'mean',
        'Season_Autumn': 'mean',
        'Season_Spring': 'mean',
        'Season_Summer': 'mean',
        'Season_Winter': 'mean',
        'Renewable_Pct': 'mean',
        'Fossil_Pct': 'mean',
        'Demand_Prev_Hour': 'mean',
        'Demand_Yesterday_Same_Hour': 'mean',
        'Demand_Last_Week_Same_Hour': 'mean',
        'Rolling_Mean_3H': 'mean',
        'Rolling_Mean_24H': 'mean'
    })

    features = daily_data[[  # Same features
        'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
        'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
        'Renewable_Pct', 'Fossil_Pct',
        'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
        'Rolling_Mean_3H', 'Rolling_Mean_24H'
    ]]
    targets = daily_data[['Demand', 'CO2_Total_Emissions']]

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(features)
    y_scaled = scaler_y.fit_transform(targets)

    X_seq, y_seq = create_sequences(X_scaled, y_scaled)
    lookback = 30
    split = int(0.7 * len(X_seq))
    X_train, X_test = X_seq[:split], X_seq[split:]
    y_train, y_test = y_seq[:split], y_seq[split:]

    model = Sequential([
        SimpleRNN(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)
    test_dates = daily_data.index[lookback + split:]

    # Compute metrics
    result = {
        'Region': region_name,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2': mape(y_true[:, 1], y_pred[:, 1])
    }
    results.append(result)

    # Save results as DataFrame
    rnn_results_df = pd.DataFrame({
        'Date': test_dates,
        'Region': region_name,
        'Model': 'RNN',
        'Actual_Demand': y_true[:, 0],
        'Predicted_Demand': y_pred[:, 0],
        'Actual_CO2': y_true[:, 1],
        'Predicted_CO2': y_pred[:, 1]
    })
    rnn_results_df.to_csv(f'rnn_{region_name.lower()}_forecast.csv', index=False)
    print(f"Saved results for {region_name}")

# Show results table
results_df = pd.DataFrame(results)
print('\nRNN Forecasting Results:')
print(results_df.round(2))


Epoch 1/10


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0257
Epoch 2/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0057
Epoch 3/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0047
Epoch 4/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0045
Epoch 5/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0043
Epoch 6/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0041
Epoch 7/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0039
Epoch 8/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0039
Epoch 9/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0037
Epoch 10/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0036
411/411 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Saved results for California
Epoch 1/10


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0417
Epoch 2/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0060
Epoch 3/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0050
Epoch 4/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0047
Epoch 5/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0044
Epoch 6/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0041
Epoch 7/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0040
Epoch 8/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0039
Epoch 9/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0038
Epoch 10/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0036
411/411 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Saved results for Texas

RNN Forecasting Results:
       Region  MAE_Demand  RMSE_Demand  MAPE_Demand  MAE_CO2  RMSE_CO2  \
0  California     1604.00      2037.28         5.21   508.32    657.08   
1       Texas     3304.38      4199.38         6.49  1315.33   1638.75   

   MAPE_CO2  
0     10.36 

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# 1. Utility functions
def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

# 2. Read dataset
df = pd.read_csv(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\featue_engg_data1.csv")
df['Date'] = pd.to_datetime(df['Date'])

# 3. Loop over regions
results = []
for region_code, region_name in [('CAL', 'California'), ('TEX', 'Texas')]:
    region_df = df[df['Region'] == region_code].copy()
    region_df.set_index('Date', inplace=True)
    daily_data = region_df.groupby(region_df.index).agg({
        'Demand': 'sum',
        'CO2_Total_Emissions': 'sum',
        'Hour': 'mean',
        'Month': 'mean',
        'DayOfWeek': 'mean',
        'Is_Weekend': 'mean',
        'DayOfYear': 'mean',
        'WeekOfYear': 'mean',
        'Season_Autumn': 'mean',
        'Season_Spring': 'mean',
        'Season_Summer': 'mean',
        'Season_Winter': 'mean',
        'Renewable_Pct': 'mean',
        'Fossil_Pct': 'mean',
        'Demand_Prev_Hour': 'mean',
        'Demand_Yesterday_Same_Hour': 'mean',
        'Demand_Last_Week_Same_Hour': 'mean',
        'Rolling_Mean_3H': 'mean',
        'Rolling_Mean_24H': 'mean'
    })

    features = daily_data[[  # Same features
        'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
        'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
        'Renewable_Pct', 'Fossil_Pct',
        'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
        'Rolling_Mean_3H', 'Rolling_Mean_24H'
    ]]
    targets = daily_data[['Demand', 'CO2_Total_Emissions']]

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(features)
    y_scaled = scaler_y.fit_transform(targets)

    X_seq, y_seq = create_sequences(X_scaled, y_scaled)
    lookback = 30
    split = int(0.65 * len(X_seq))
    X_train, X_test = X_seq[:split], X_seq[split:]
    y_train, y_test = y_seq[:split], y_seq[split:]

    model = Sequential([
        SimpleRNN(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)
    test_dates = daily_data.index[lookback + split:]

    # Compute metrics
    result = {
        'Region': region_name,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2': mape(y_true[:, 1], y_pred[:, 1])
    }
    results.append(result)

    # Save results as DataFrame
    rnn_results_df = pd.DataFrame({
        'Date': test_dates,
        'Region': region_name,
        'Model': 'RNN',
        'Actual_Demand': y_true[:, 0],
        'Predicted_Demand': y_pred[:, 0],
        'Actual_CO2': y_true[:, 1],
        'Predicted_CO2': y_pred[:, 1]
    })
    rnn_results_df.to_csv(f'rnn_{region_name.lower()}_forecast.csv', index=False)
    print(f"Saved results for {region_name}")

# Show results table
results_df = pd.DataFrame(results)
print('\nRNN Forecasting Results:')
print(results_df.round(2))


Epoch 1/10


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


445/445 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0230
Epoch 2/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0065
Epoch 3/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0054
Epoch 4/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0050
Epoch 5/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0047
Epoch 6/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0044
Epoch 7/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0042
Epoch 8/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0040
Epoch 9/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0038
Epoch 10/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0039
479/479 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Saved results for California
Epoch 1/10


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


445/445 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0467
Epoch 2/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0056
Epoch 3/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0049
Epoch 4/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0046
Epoch 5/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0042
Epoch 6/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0043
Epoch 7/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0041
Epoch 8/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0039
Epoch 9/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0039
Epoch 10/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0038
479/479 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Saved results for Texas

RNN Forecasting Results:
       Region  MAE_Demand  RMSE_Demand  MAPE_Demand  MAE_CO2  RMSE_CO2  \
0  California     1380.25      1780.28         4.54   486.21    609.31   
1       Texas     3303.87      4104.13         6.73  1309.77   1680.49   

   MAPE_CO2  
0     10.77 

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# 1. Utility functions
def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

# 2. Read dataset
df = pd.read_csv(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\featue_engg_data1.csv")
df['Date'] = pd.to_datetime(df['Date'])

# 3. Loop over regions
results = []
for region_code, region_name in [('CAL', 'California'), ('TEX', 'Texas')]:
    region_df = df[df['Region'] == region_code].copy()
    region_df.set_index('Date', inplace=True)
    daily_data = region_df.groupby(region_df.index).agg({
        'Demand': 'sum',
        'CO2_Total_Emissions': 'sum',
        'Hour': 'mean',
        'Month': 'mean',
        'DayOfWeek': 'mean',
        'Is_Weekend': 'mean',
        'DayOfYear': 'mean',
        'WeekOfYear': 'mean',
        'Season_Autumn': 'mean',
        'Season_Spring': 'mean',
        'Season_Summer': 'mean',
        'Season_Winter': 'mean',
        'Renewable_Pct': 'mean',
        'Fossil_Pct': 'mean',
        'Demand_Prev_Hour': 'mean',
        'Demand_Yesterday_Same_Hour': 'mean',
        'Demand_Last_Week_Same_Hour': 'mean',
        'Rolling_Mean_3H': 'mean',
        'Rolling_Mean_24H': 'mean'
    })

    features = daily_data[[  # Same features
        'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
        'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
        'Renewable_Pct', 'Fossil_Pct',
        'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
        'Rolling_Mean_3H', 'Rolling_Mean_24H'
    ]]
    targets = daily_data[['Demand', 'CO2_Total_Emissions']]

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(features)
    y_scaled = scaler_y.fit_transform(targets)

    X_seq, y_seq = create_sequences(X_scaled, y_scaled)
    lookback = 30
    split = int(0.6 * len(X_seq))
    X_train, X_test = X_seq[:split], X_seq[split:]
    y_train, y_test = y_seq[:split], y_seq[split:]

    model = Sequential([
        SimpleRNN(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)
    test_dates = daily_data.index[lookback + split:]

    # Compute metrics
    result = {
        'Region': region_name,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2': mape(y_true[:, 1], y_pred[:, 1])
    }
    results.append(result)

    # Save results as DataFrame
    rnn_results_df = pd.DataFrame({
        'Date': test_dates,
        'Region': region_name,
        'Model': 'RNN',
        'Actual_Demand': y_true[:, 0],
        'Predicted_Demand': y_pred[:, 0],
        'Actual_CO2': y_true[:, 1],
        'Predicted_CO2': y_pred[:, 1]
    })
    rnn_results_df.to_csv(f'rnn_{region_name.lower()}_forecast.csv', index=False)
    print(f"Saved results for {region_name}")

# Show results table
results_df = pd.DataFrame(results)
print('\nRNN Forecasting Results:')
print(results_df.round(2))


Epoch 1/10


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0673
Epoch 2/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0075
Epoch 3/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0064
Epoch 4/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0058
Epoch 5/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0048
Epoch 6/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0047
Epoch 7/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0042
Epoch 8/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0040
Epoch 9/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0041
Epoch 10/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0038
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Saved results for California


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.0273
Epoch 2/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0061
Epoch 3/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0053
Epoch 4/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0047
Epoch 5/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0047
Epoch 6/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0044
Epoch 7/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0042
Epoch 8/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0040
Epoch 9/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0038
Epoch 10/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0039
547/547 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Saved results for Texas

RNN Forecasting Results:
       Region  MAE_Demand  RMSE_Demand  MAPE_Demand  MAE_CO2  RMSE_CO2  \
0  California     1558.59      1982.83         4.98   516.73    686.34   
1       Texas     3300.29      4136.89         6.78  1447.66   1889.57   

   MAPE_CO2  
0